<a href="https://colab.research.google.com/github/daniolivaglez/03MAIR-Algoritmos-De-Optimizacion/blob/main/AG3/Daniel_Oliva_AG3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
AG3- Actividad Guiada 3
Nombre: Daniel Oliva González
https://colab.research.google.com/drive/1H4hge994trpr4LVTD5u6Jbx6khRS9Z3H?usp=sharing